In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re

### Download ISO codes

In [3]:

#functions
def gdp(x):

    if  "Trillion" in x:
        return float(x.replace("Trillion",""))*1000
    elif  "Billion" in x:
        return float(x.replace("Billion",""))
    elif "Million" in x:
        return float(x.replace("Million",""))/1000

#scraping
url="https://www.iban.com/country-codes"
page=get(url)

html_soup = BeautifulSoup(page.text, 'html.parser')


lista = []
kislista = []
for idx,val in enumerate(html_soup.find_all('td'),1):
    kislista.append(str(val)[4:-5])
    if len(kislista)==4:
        lista.append(kislista)
        kislista = []


isos = pd.DataFrame(lista)


In [4]:
isos_h = isos.iloc[:,[1,2]].set_index([2]).T.to_dict()

In [5]:
def dist(x):
    try:
        return isos_h[x][1]
    except:
        return x

In [6]:
dist_data = pd.read_excel("../../output/dist_cepii.xls")[["iso_o","iso_d","dist"]]

In [7]:
dist_data["iso_o"] = dist_data["iso_o"].apply(dist)
dist_data["iso_d"] = dist_data["iso_d"].apply(dist)

In [8]:
dist_data.sample(3)

,iso_o,iso_d,dist
14524,FI,SZ,9639.990
3022,AZ,LA,5599.605
7004,BW,EH,7140.941


In [9]:
TCI = pd.read_csv("../../output/TC_INDEX_EXP_.csv", index_col=0)#.set_index(["DECLARANT","PARTNER"])

In [10]:
for col in range(2001,2018):
    
    TCI[str(col)] = col

#TCI[str(asd) for asd in range(2001,2019)] = col for col in range(2001,2019)

In [11]:
GDPs = pd.read_csv("../../output/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_10576699.csv")

In [12]:
cc_name_dict = pd.read_csv("../../output/CC_detail.csv", index_col=0)[["c_name","ISO_code"]].set_index("c_name").T.to_dict()

In [13]:
def cc_name_ISO(x):
    try:
        return cc_name_dict[x]["ISO_code"]
    except:
        return x

In [14]:
GDPs["Country Name"] = GDPs["Country Name"].apply(cc_name_ISO)

In [15]:
for col in range(2001,2018):
    
    GDPs[str(col)+"_y"] = col

In [17]:
MERGED_01 =  pd.merge(
    pd.lreshape(TCI, {'TCI':["TCI_"+str(year) for year in range(2001,2018)],
                  'year':[str(year) for year in range(2001,2018)]})
    ,dist_data[["iso_o","iso_d","dist"]]
    ,how="left", left_on=["DECLARANT","PARTNER"], right_on=["iso_o","iso_d"])\
    .drop(["iso_o","iso_d"],axis=1)


In [18]:
MERGED_02 = pd.merge(
         MERGED_01, pd.lreshape(GDPs, {'gdp':[str(year) for year in range(2001,2018)],
                                       'year':[str(year)+"_y" for year in range(2001,2018)]}).drop(["2000"],axis=1).pipe(lambda df: df.assign(ln_GDP = df["gdp"].apply(pd.np.log)))\
         ,how="left", left_on=["DECLARANT","year"], right_on=["Country Name","year"]).drop("Country Name",axis=1).rename(columns = {"ln_GDP":"ln_gdp_DEC"}).drop(["gdp"],axis=1)

In [19]:
MERGED_02.sample(3)

,DECLARANT,PARTNER,TCI,year,dist,ln_gdp_DEC
62894,RO,HT,2.572038,2012,NaN,9.051936
58358,AT,GW,2.044192,2012,5016.396,10.790714
57812,SE,IN,0.755606,2011,5574.222,10.996519


In [22]:
pd.lreshape(GDPs, {'gdp':[str(year) for year in range(2001,2018)]
                                              ,'year':[str(year)+"_y" for year in range(2001,2018)]})\
                   .drop(["2000"],axis=1).sample(3)

,Country Name,gdp,year
3972,US,57904.204329,2016
4101,Lao PDR,2457.377414,2017
1662,NL,51241.315617,2007


In [23]:
MERGED_03= pd.merge(MERGED_02,
                    pd.lreshape(GDPs, {'gdp':[str(year) for year in range(2001,2018)]
                                              ,'year':[str(year)+"_y" for year in range(2001,2018)]}).drop(["2000"],axis=1).pipe(lambda df: df.assign(ln_GDP = df["gdp"].apply(pd.np.log)))\
                   ,how="left"\
                   ,left_on = ["PARTNER","year"]\
                   ,right_on=["Country Name","year"])\
.drop("Country Name",axis=1).rename(columns = {"ln_GDP":"ln_gdp_PART"}).drop(["gdp"],axis=1)


In [24]:
len(MERGED_03),len(dist_data),len(GDPs)

(90414, 50176, 264)

In [25]:
MERGED_03.isna().sum()

DECLARANT          0
PARTNER            0
TCI                0
year               0
dist           11632
ln_gdp_DEC      3086
ln_gdp_PART    24937
dtype: int64

In [26]:
MERGED_03.dropna().iloc[:,2:].corr()

,TCI,year,dist,ln_gdp_DEC,ln_gdp_PART
TCI,1.000000,-0.004791,0.099180,-0.209145,-0.060925
year,-0.004791,1.000000,0.037354,0.277811,0.191740
dist,0.099180,0.037354,1.000000,0.043054,-0.003550
ln_gdp_DEC,-0.209145,0.277811,0.043054,1.000000,0.065710
ln_gdp_PART,-0.060925,0.191740,-0.003550,0.065710,1.000000


In [27]:
pop =  pd.read_csv("../../output/API_SP.POP.TOTL_DS2_en_csv_v2_10576638.csv")

In [28]:
pop["Country Name"] = pop["Country Name"].apply(cc_name_ISO)

In [29]:
pop

,Country Name,Country Code,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,AW,ABW,9.289800e+04,9.499200e+04,9.701700e+04,9.873700e+04,1.000310e+05,1.008320e+05,1.012200e+05,1.013530e+05,1.014530e+05,1.016690e+05,1.020530e+05,1.025770e+05,1.031870e+05,1.037950e+05,1.043410e+05,1.048220e+05,1.052640e+05
1,AF,AFG,2.096646e+07,2.197992e+07,2.306485e+07,2.411898e+07,2.507080e+07,2.589345e+07,2.661679e+07,2.729403e+07,2.800433e+07,2.880317e+07,2.970860e+07,3.069696e+07,3.173169e+07,3.275802e+07,3.373649e+07,3.465603e+07,3.553008e+07
2,AO,AGO,1.698327e+07,1.757265e+07,1.820337e+07,1.886572e+07,1.955254e+07,2.026240e+07,2.099769e+07,2.175942e+07,2.254955e+07,2.336913e+07,2.421856e+07,2.509615e+07,2.599834e+07,2.692047e+07,2.785930e+07,2.881346e+07,2.978419e+07
3,AL,ALB,3.060173e+06,3.051010e+06,3.039616e+06,3.026939e+06,3.011487e+06,2.992547e+06,2.970017e+06,2.947314e+06,2.927519e+06,2.913021e+06,2.905195e+06,2.900401e+06,2.895092e+06,2.889104e+06,2.880703e+06,2.876101e+06,2.873457e+06
4,AD,AND,6.734100e+04,7.004900e+04,7.318200e+04,7.624400e+04,7.886700e+04,8.099100e+04,8.268300e+04,8.386100e+04,8.446200e+04,8.444900e+04,8.375100e+04,8.243100e+04,8.078800e+04,7.922300e+04,7.801400e+04,7.728100e+04,7.696500e+04
5,Arab World,ARB,2.898504e+08,2.960266e+08,3.024345e+08,3.091620e+08,3.162647e+08,3.237733e+08,3.316538e+08,3.398255e+08,3.481451e+08,3.565089e+08,3.648959e+08,3.733070e+08,3.817021e+08,3.900430e+08,3.983050e+08,4.064527e+08,4.144919e+08
6,AE,ARE,3.326032e+06,3.507232e+06,3.741932e+06,4.087931e+06,4.579562e+06,5.242032e+06,6.044067e+06,6.894278e+06,7.666393e+06,8.270684e+06,8.672475e+06,8.900453e+06,9.006263e+06,9.070867e+06,9.154302e+06,9.269612e+06,9.400145e+06
7,AR,ARG,3.747151e+07,3.788937e+07,3.830938e+07,3.872870e+07,3.914549e+07,3.955889e+07,3.997022e+07,4.038239e+07,4.079941e+07,4.122389e+07,4.165688e+07,4.209674e+07,4.253992e+07,4.298152e+07,4.341776e+07,4.384743e+07,4.427104e+07
8,AM,ARM,3.050655e+06,3.033897e+06,3.017806e+06,3.000612e+06,2.981259e+06,2.958500e+06,2.933056e+06,2.908220e+06,2.888584e+06,2.877311e+06,2.875581e+06,2.881922e+06,2.893509e+06,2.906220e+06,2.916950e+06,2.924816e+06,2.930450e+06
9,AS,ASM,5.817500e+04,5.873100e+04,5.911700e+04,5.926400e+04,5.911800e+04,5.865000e+04,5.790300e+04,5.703000e+04,5.622700e+04,5.563700e+04,5.532000e+04,5.523000e+04,5.530700e+04,5.543700e+04,5.553700e+04,5.559900e+04,5.564100e+04


In [30]:
for col in range(2001,2018):
    
    pop[str(col)+"_y"] = col

In [31]:
MERGED_04 = pd.merge(MERGED_03,
                     pd.lreshape(pop, {'pop':[str(year) for year in range(2001,2018)],
                                       'year':[str(year)+"_y" for year in range(2001,2018)]})\
                     .pipe(lambda df: df.assign(ln_pop = df["pop"].apply(pd.np.log)))[["Country Name","year","ln_pop"]]
        ,how="left"\
        ,left_on = ["PARTNER","year"]\
        ,right_on=["Country Name","year"])\
.drop("Country Name",axis=1)\
.rename(columns = {"ln_pop":"ln_pop_PART"})


In [37]:
MERGED_04.head()

,DECLARANT,PARTNER,TCI,year,dist,ln_gdp_DEC,ln_gdp_PART,ln_pop_PART
0,AT,AD,1.708595,2001,1322.201,10.107958,10.009146,11.117525
1,AT,AE,0.887546,2001,4245.842,10.107958,10.343725,15.017291
2,AT,AF,2.466641,2001,4568.098,10.107958,NaN,16.858435
3,AT,AG,2.439151,2001,7737.498,10.107958,9.149970,11.351077
4,AT,AI,2.392736,2001,7742.359,10.107958,NaN,NaN


In [32]:
MERGED_05 = pd.merge(MERGED_04,
                     pd.lreshape(pop, {'pop':[str(year) for year in range(2001,2018)],
                                       'year':[str(year)+"_y" for year in range(2001,2018)]})\
                     .pipe(lambda df: df.assign(ln_pop = df["pop"].apply(pd.np.log)))[["Country Name","year","ln_pop"]]
        ,how="left"\
        ,left_on = ["DECLARANT","year"]\
        ,right_on=["Country Name","year"])\
.drop("Country Name",axis=1)\
.rename(columns = {"ln_pop":"ln_pop_DEC"})

In [42]:
MERGED_05.head()

,DECLARANT,PARTNER,TCI,year,dist,ln_gdp_DEC,ln_gdp_PART,ln_pop_PART,ln_pop_DEC
0,AT,AD,1.708595,2001,1322.201,10.107958,10.009146,11.117525,15.900225
1,AT,AE,0.887546,2001,4245.842,10.107958,10.343725,15.017291,15.900225
2,AT,AF,2.466641,2001,4568.098,10.107958,NaN,16.858435,15.900225
3,AT,AG,2.439151,2001,7737.498,10.107958,9.149970,11.351077,15.900225
4,AT,AI,2.392736,2001,7742.359,10.107958,NaN,NaN,15.900225


In [55]:
dec_dums = pd.get_dummies(MERGED_05.DECLARANT, dummy_na=True)
dec_dums.columns = ["DEC_"+str(x) for x in pd.get_dummies(MERGED_05.DECLARANT, dummy_na=True).columns]

par_dums = pd.get_dummies(MERGED_05.PARTNER, dummy_na=True)
par_dums.columns = ["PAR_"+str(x) for x in pd.get_dummies(MERGED_05.PARTNER, dummy_na=True).columns]

In [60]:
result = pd.concat([MERGED_05, dec_dums,par_dums], axis=1, sort=False).drop(["DECLARANT","PARTNER"], axis = 1)

In [79]:
result["log_dist"]=result["dist"].apply(pd.np.log)

AttributeError: module 'numpy' has no attribute 'ln'

In [75]:
import statsmodels.api as sm

In [76]:
result = result.dropna()
y = result["TCI"]
x = result.drop(["TCI"],axis=1)

In [77]:
len(result),len(result)

(59110, 59110)

In [78]:
# Fit and summarize OLS model
model = sm.OLS(y, x).fit()

model.summary()

/home/berei/.local/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/berei/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/berei/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/berei/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    TCI   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.590
Method:                 Least Squares   F-statistic:                     465.2
Date:                Tue, 04 Jun 2019   Prob (F-statistic):               0.00
Time:                        15:04:48   Log-Likelihood:                -63006.
No. Observations:               59110   AIC:                         1.264e+05
Df Residuals:                   58926   BIC:                         1.280e+05
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
year           -0.0043      0.001     -3.250      0.001      -0.007      -0.002
dist            0.5675      0.116      4.897      0.000       0.340       0.795
ln_gdp_DEC      0.0334      0.016      2.086      0.037       0.002       0.065
ln_gdp_PART     0.0319      0.013      2.460      0.014       0.006       0.057
ln_pop_PART    -0.2974      0.039     -7.534      0.000      -0.375      -0.220
ln_pop_DEC      0.4507      0.085      5.296      0.000       0.284       0.618
DEC_AT          1.0602      2.089      0.507      0.612      -3.035       5.156
DEC_BE          0.1760      2.096      0.084      0.933      -3.933       4.285
DEC_BG          1.7601      2.090      0.842      0.400      -2.336       5.857
DEC_CY          2.7383      2.045      1.339      0.181      -1.270       6.747
DEC_CZ          1.1507      2.097      0.549      0.583      -2.959       5.260
DEC_DE         -1.1069      2.156     -0.513      0.608      -5.333       3.119
DEC_DK          0.9832      2.079      0.473      0.636      -3.091       5.057
DEC_EE          2.8208      2.050      1.376      0.169      -1.197       6.839
DEC_ES         -0.4585      2.138     -0.215      0.830      -4.648       3.731
DEC_FI          1.3201      2.078      0.635      0.525      -2.753       5.393
DEC_FR         -0.9124      2.148     -0.425      0.671      -5.123       3.298
DEC_GB         -0.8319      2.147     -0.387      0.698      -5.040       3.376
DEC_GR          1.0643      2.098      0.507      0.612      -3.047       5.176
DEC_HR          1.9406      2.075      0.935      0.350      -2.127       6.008
DEC_HU          1.5078      2.096      0.719      0.472      -2.601       5.616
DEC_IE          1.5905      2.073      0.767      0.443      -2.473       5.654
DEC_IT         -0.7242      2.146     -0.337      0.736      -4.930       3.482
DEC_LT          2.2230      2.068      1.075      0.282      -1.831       6.277
DEC_LU          3.5775      2.029      1.763      0.078      -0.399       7.554
DEC_LV          2.5727      2.060      1.249      0.212      -1.465       6.610
DEC_MT          3.4653      2.029      1.708      0.088      -0.512       7.443
DEC_NL         -0.2045      2.108     -0.097      0.923      -4.335       3.926
DEC_PL          0.2683      2.134      0.126      0.900      -3.914       4.450
DEC_PT          0.8991      2.096      0.429      0.668      -3.210       5.008
DEC_RO       1.975e-13   4.61e-14      4.281      0.000    1.07e-13    2.88e-13
DEC_SE          0.7148      2.092      0.342      0.733      -3.385       4.815
DEC_SI          2.3635      2.058      1.149      0.251      -1.670       6.397
DEC_SK       2.863e-13   3.16e-13      0.907      0.364   -3.32e-13    9.05e-13
DEC_nan      4.152e-14    1.7e-13      0.244      0.808   -2.93e-13    3.76e-13
PAR_..       8.705e-14   2.11e-13      0.41